# Парсинг текстов с сайтов и соцсетей

Этот блокнот загружает скрипты из репозитория и помогает быстро получить тексты из:
- статей и страниц сайтов (универсальный парсер)
- отзывов Яндекса
- групп ВК

Ниже — установка зависимостей, импорт функций и интерактивные формы для ввода URL/токенов.


In [ ]:
#@title Установка зависимостей
!pip -q install requests beautifulsoup4 lxml ipywidgets pandas shapely pymorphy3 flair folium tqdm

import os
import sys
from pathlib import Path

# Укажите репозиторий в формате "owner/repo" при необходимости
REPO_SLUG = os.environ.get("GITHUB_REPO", "Sandrro/digital_identity")
REPO_URL = f"https://github.com/{REPO_SLUG}"
REPO_DIR = Path("/content/digital_identity")

if not REPO_DIR.exists():
    !git clone --depth 1 {REPO_URL} {REPO_DIR}

sys.path.insert(0, str(REPO_DIR))


In [ ]:
#@title Импорт парсеров
import sys
sys.path.insert(0, "/content/digital_identity/parsers")
from vk_group_parser import VKGroupParser
from website_parser import parse_websites
from yandex_reviews_parser import fetch_yandex_reviews


In [ ]:
#@title Интерактивный запуск
import csv
from datetime import date, datetime, timezone
from pathlib import Path

import ipywidgets as widgets
import pandas as pd
from IPython.display import display

OUTPUT_CSV = Path("/content/digital_identity/parsed_texts.csv")

parser_dropdown = widgets.Dropdown(
    options=[
        ("Сайты", "website"),
        ("Яндекс отзывы", "yandex"),
        ("ВК группа", "vk"),
    ],
    description="Парсер:"
)
url_input = widgets.Text(
    value="",
    description="URL/домен:",
    placeholder="https://...",
    layout=widgets.Layout(width="80%"),
)
token_input = widgets.Password(
    value="",
    description="VK токен:",
    placeholder="Требуется только для ВК",
)
max_items = widgets.IntSlider(
    value=20,
    min=1,
    max=200,
    step=1,
    description="Лимит:"
)
run_button = widgets.Button(description="Запустить")
output = widgets.Output()

def _normalize_date(value):
    if value in (None, ""):
        return None
    if isinstance(value, datetime):
        return value.date().isoformat()
    if isinstance(value, date):
        return value.isoformat()
    parsed = pd.to_datetime(value, errors="coerce", utc=True)
    if pd.isna(parsed):
        return str(value)[:10]
    return parsed.date().isoformat()

def _format_vk_timestamp(ts):
    if ts is None:
        return None
    return datetime.fromtimestamp(ts, tz=timezone.utc).date().isoformat()

def _write_rows(rows):
    if not rows:
        return
    OUTPUT_CSV.parent.mkdir(parents=True, exist_ok=True)
    with OUTPUT_CSV.open("w", newline="", encoding="utf-8") as handle:
        writer = csv.DictWriter(
            handle,
            fieldnames=["source", "url", "title", "timestamp", "text"],
        )
        writer.writeheader()
        writer.writerows(rows)

def run_parser(_):
    output.clear_output()
    with output:
        rows = []
        parser = parser_dropdown.value
        if parser == "website":
            df = parse_websites([url_input.value])
            for _, row in df.iterrows():
                meta = row.get("meta") or {}
                rows.append({
                    "source": row.get("source"),
                    "url": row.get("url"),
                    "title": meta.get("title"),
                    "timestamp": _normalize_date(row.get("date")),
                    "text": row.get("text_clean") or row.get("text_raw"),
                })
        elif parser == "yandex":
            reviews = fetch_yandex_reviews(url_input.value, max_reviews=max_items.value)
            for review in reviews:
                rows.append({
                    "source": "yandex",
                    "url": url_input.value,
                    "title": None,
                    "timestamp": None,
                    "text": review.text,
                })
        else:
            if not token_input.value:
                raise ValueError("Нужен VK токен.")
            vk = VKGroupParser(token_input.value)
            for post in vk.iter_posts(url_input.value, total=max_items.value):
                rows.append({
                    "source": "vk",
                    "url": url_input.value,
                    "title": None,
                    "timestamp": _format_vk_timestamp(post.date),
                    "text": post.text,
                })
        _write_rows(rows)
        if rows:
            display(pd.DataFrame(rows))

run_button.on_click(run_parser)
display(parser_dropdown, url_input, token_input, max_items, run_button, output)


## Геокодирование и визуализация адресов

Ниже добавлен импорт скрипта геокодера и UI для запуска.
В качестве входа используется файл, сохранённый в предыдущей ячейке
(`parsed_texts.csv`). Скрипт извлекает топонимы, геокодирует их через Photon,
сохраняет результаты в GeoJSON и строит интерактивную карту.


In [ ]:
#@title Геокодер: импорт и запуск с UI
import asyncio
from pathlib import Path

import ipywidgets as widgets
import pandas as pd
from IPython.display import display

import sys
sys.path.insert(0, "/content/digital_identity/parsers")
from geocoder import geocode_texts, build_geojson, save_geojson, save_map

DEFAULT_INPUT = Path("/content/digital_identity/parsed_texts.csv")

file_picker = widgets.Text(
    value=str(DEFAULT_INPUT),
    description="CSV файл:",
    layout=widgets.Layout(width="80%"),
)
bbox_input = widgets.Text(
    value="",
    description="BBox:",
    placeholder="minx,miny,maxx,maxy (необязательно)",
    layout=widgets.Layout(width="80%"),
)
geojson_output = widgets.Text(
    value="/content/digital_identity/geocoded_points.geojson",
    description="GeoJSON:",
    layout=widgets.Layout(width="80%"),
)
map_output = widgets.Text(
    value="/content/digital_identity/geocoded_points.html",
    description="HTML карта:",
    layout=widgets.Layout(width="80%"),
)
run_geocoder = widgets.Button(description="Геокодировать")
geo_output = widgets.Output()

def _load_texts(csv_path: Path) -> list[str]:
    df = pd.read_csv(csv_path)
    if "text" not in df.columns:
        raise ValueError("CSV должен содержать колонку text")
    return df["text"].fillna("").astype(str).tolist()

def _run_geocoding(_):
    geo_output.clear_output()
    with geo_output:
        csv_path = Path(file_picker.value).expanduser()
        if not csv_path.exists():
            raise FileNotFoundError(f"Файл не найден: {csv_path}")
        texts = _load_texts(csv_path)
        bbox = bbox_input.value.strip() or None
        results = asyncio.run(geocode_texts(texts, bbox=bbox))
        geojson_data = build_geojson(results)
        save_geojson(geojson_output.value, geojson_data)
        save_map(map_output.value, results)
        display({"geojson": geojson_output.value, "map": map_output.value})

run_geocoder.on_click(_run_geocoding)
display(file_picker, bbox_input, geojson_output, map_output, run_geocoder, geo_output)
